# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import string
import sys
import re
import pickle

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
import pickle


import warnings
warnings.simplefilter('ignore')

In [2]:
import nltk
nltk.download(['punkt','wordnet','stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
Y = df.drop(['id','message','original','genre'], axis=1)
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [5]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
remove_punctu = str.maketrans('','', string.punctuation)

In [6]:
def tokenize(text):
    #tokenize text
    tokens = nltk.word_tokenize(text)
    #Lemmatize and removing stop words
    return[lemmatizer.lemmatize(word).lower().strip() for word in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultiOutputClassifier(RandomForestClassifier()))
                   ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
#predict on the test data
y_pred = pipeline.predict(X_test)

#classification report
def each_col_report(Y_test, y_pred):
    for i, col in enumerate(Y_test.columns): 
        print('---------------------------------')
        print(col)
        print(classification_report(Y_test.iloc[:,i], y_pred[:,i]))

each_col_report(Y_test, y_pred)

---------------------------------
related
             precision    recall  f1-score   support

        0.0       0.34      0.13      0.18      1604
        1.0       0.76      0.92      0.83      4906

avg / total       0.66      0.72      0.67      6510

---------------------------------
request
             precision    recall  f1-score   support

        0.0       0.84      0.98      0.91      5412
        1.0       0.48      0.08      0.13      1098

avg / total       0.78      0.83      0.78      6510

---------------------------------
offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6483
        1.0       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6510

---------------------------------
aid_related
             precision    recall  f1-score   support

        0.0       0.60      0.83      0.70      3872
        1.0       0.45      0.20      0.27      2638

avg / total       0.54  

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
    'tfidf__norm':['l2','l1'],
    'clf__estimator__min_samples_split':[2,3],
    }

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__norm': ['l2', 'l1'], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
y_pred = cv.predict(X_test)

In [13]:
accuracy = (y_pred == Y_test).sum()/Y_test.shape[0]
print("accuracy :", accuracy)
each_col_report(Y_test, y_pred)

accuracy : related                   0.723349
request                   0.829186
offer                     0.995853
aid_related               0.579109
medical_help              0.920430
medical_products          0.951459
search_and_rescue         0.970968
security                  0.984025
military                  0.968049
child_alone               1.000000
water                     0.933641
food                      0.887558
shelter                   0.905223
clothing                  0.987250
money                     0.978495
missing_people            0.989094
refugees                  0.970507
death                     0.952535
other_aid                 0.867127
infrastructure_related    0.935637
transport                 0.951306
buildings                 0.944086
electricity               0.979877
tools                     0.994163
hospitals                 0.988786
shops                     0.995545
aid_centers               0.987711
other_infrastructure      0.957911
weather_r

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
improve_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 100)))
    ])


improve_pipeline.fit(X_train, Y_train)
y_pred_improve = improve_pipeline.predict(X_test)
each_col_report(Y_test, y_pred_improve)

---------------------------------
related
             precision    recall  f1-score   support

        0.0       0.34      0.05      0.09      1604
        1.0       0.76      0.97      0.85      4906

avg / total       0.66      0.74      0.66      6510

---------------------------------
request
             precision    recall  f1-score   support

        0.0       0.84      0.96      0.90      5412
        1.0       0.39      0.12      0.19      1098

avg / total       0.77      0.82      0.78      6510

---------------------------------
offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6483
        1.0       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      6510

---------------------------------
aid_related
             precision    recall  f1-score   support

        0.0       0.60      0.85      0.71      3872
        1.0       0.45      0.17      0.25      2638

avg / total       0.54  

### 9. Export your model as a pickle file

In [27]:
with open("disaster_model.pk1", 'wb') as pickle_file:
    pickle.dump(improve_pipeline, pickle_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.